# Bond Calculator for YTM, Duration and Convexity

The purpose of this project is to find price of bonds given time, payment frequency, interest rate, payment amount, and future value

After finding YTM we will also show the bonds Duration using the first derivative of the bond equation and the Convexity using the second derivative of the equation

In [4]:
#Import Data for sampling:
bonds = [
    {"Bond": "B001", "FV": 1000, "C": 0.05, "T": 10, "PaymentFrequency": 2, "YTM": 0.06}, 
    {"Bond": "B002", "FV": 1000, "C": 0.03, "T": 5, "PaymentFrequency": 1, "YTM": 0.04},  
    {"Bond": "B003", "FV": 1000, "C": 0.07, "T": 15, "PaymentFrequency": 2, "YTM": 0.05},  
]

This is the equation for bond prices. C represents the amount per cash flow, r is the interest rate or YTM, t is the time of current period, T is the total numbe of periods, FV is the future value and P is the price
$$
\
P(r) = \sum_{t=1}^{T} \frac{C}{(1+r)^t} + \frac{FV}{(1+r)^T}
\
$$

In [6]:
#Bond calculation Function
def calculate_bond_price(FV, C, T, YTM, PaymentFrequency):
   
    # Calculate coupon payment per period
    coupon_payment = C * FV / PaymentFrequency
    
    # Total number of periods
    num_periods = T * PaymentFrequency
    
    # Periodic yield
    periodic_yield = YTM / PaymentFrequency
    
    # Calculate price as the sum of discounted cash flows
    price = sum(coupon_payment / (1 + periodic_yield)**t for t in range(1, num_periods + 1))
    
    # Add the discounted face value
    price += FV / (1 + periodic_yield)**num_periods
    
    return price

In [7]:
# Calculate and print bond prices
for bond in bonds:
    price = calculate_bond_price(
        FV=bond["FV"],
        C=bond["C"],
        T=bond["T"],
        YTM=bond["YTM"],
        PaymentFrequency=bond["PaymentFrequency"]
    )
    print(f"Bond {bond['Bond']} Price: ${price:.2f}")

Bond B001 Price: $925.61
Bond B002 Price: $955.48
Bond B003 Price: $1209.30


The first derivative, which represents duration, can be written as:
$$
\
P'(r) = - \sum_{t=1}^{T} \frac{t \cdot C}{(1 + r)^{t+1}} - \frac{T \cdot FV}{(1 + r)^{T+1}}
\
$$

This gives a better representation of a bonds sensitivity to a change in interest rate change

Macaulay Duration is the weighted average time of a bond's cash flows, weighted by present value of cash flows. It is represented as: 
$$
D = \frac{1}{P} \sum_{t=1}^{T} \frac{t \cdot C}{(1+r)^t} + \frac{T \cdot FV}{(1+r)^T}
$$

In [10]:
def calculate_macaulay_duration(FV, C, T, YTM, PaymentFrequency):
    # Calculate Macaulay Duration using the formula
    price = calculate_bond_price(FV, C, T, YTM, PaymentFrequency)
    duration = 0
    for t in range(1, T + 1):
        duration += t * C / (1 + YTM / PaymentFrequency) ** (t * PaymentFrequency)
    duration += T * FV / (1 + YTM / PaymentFrequency) ** (T * PaymentFrequency)
    macaulay_duration = duration / price
    print(f"Macaulay Duration: {macaulay_duration:.2f} years")  # Print Macaulay duration
    return macaulay_duration

In [11]:
for bond in bonds:
    print(f"Calculations for {bond['Bond']}:\n")
    calculate_macaulay_duration(
        FV=bond["FV"],
        C=bond["C"],
        T=bond["T"],
        YTM=bond["YTM"],
        PaymentFrequency=bond["PaymentFrequency"]
    )

Calculations for B001:

Macaulay Duration: 5.98 years
Calculations for B002:

Macaulay Duration: 4.30 years
Calculations for B003:

Macaulay Duration: 5.92 years


Modified Duration is the macaulay duration adjusted for the bond's yield
$$
D_{\text{mod}} = \frac{D}{(1+r)}
$$

In [13]:
def calculate_modified_duration(FV, C, T, YTM, PaymentFrequency):
    # Calculate Modified Duration using the formula
    macaulay_duration = calculate_macaulay_duration(FV, C, T, YTM, PaymentFrequency)
    modified_duration = macaulay_duration / (1 + YTM / PaymentFrequency)
    print(f"Modified Duration: {modified_duration:.2f} years\n")  # Print Modified duration
    return modified_duration

In [14]:
for bond in bonds:
    print(f"Calculations for {bond['Bond']}:\n")
    calculate_modified_duration(
        FV=bond["FV"],
        C=bond["C"],
        T=bond["T"],
        YTM=bond["YTM"],
        PaymentFrequency=bond["PaymentFrequency"]
    )

Calculations for B001:

Macaulay Duration: 5.98 years
Modified Duration: 5.81 years

Calculations for B002:

Macaulay Duration: 4.30 years
Modified Duration: 4.14 years

Calculations for B003:

Macaulay Duration: 5.92 years
Modified Duration: 5.77 years



Convexity represents the curvature in the bond price-yield relationship. Duration only showcases the linear approximation, so convexity captures the non-linear relationship. It is represented by the second derivative of the bond equation
$$
\
C = \ \sum_{t=1}^{T} \frac{t(t+1) \cdot C}{(1+r)^{t+2}} + \frac{r(r+1) \cdot FV}{(1+r)^{T+2}}
\
$$

To calculate the adjustment we must normalize for price
$$
\
C = \frac{1}{P} \sum_{t=1}^{T} \frac{t(t+1) \cdot C}{(1+r)^{t+2}} + \frac{T(T+1) \cdot FV}{(1+r)^{T+2}}
\
$$

In [17]:
def calculate_convexity(FV, C, T, YTM, PaymentFrequency):
    # Calculate bond price using the bond price formula
    price = calculate_bond_price(FV, C, T, YTM, PaymentFrequency)
    convexity = 0
    for t in range(1, T + 1):
        convexity += t * (t + 1) * C / (1 + YTM / PaymentFrequency) ** (t * PaymentFrequency + 2)
    convexity += T * (T + 1) * FV / (1 + YTM / PaymentFrequency) ** (T * PaymentFrequency + 2)
    convexity = convexity / price
    print(f"Convexity: {convexity:.4f} years^2\n")  # Print Convexity
    return convexity

In [18]:
for bond in bonds:
 print(f"Calculations for {bond['Bond']}:\n")   
 calculate_convexity(
        FV=bond["FV"],
        C=bond["C"],
        T=bond["T"],
        YTM=bond["YTM"],
        PaymentFrequency=bond["PaymentFrequency"]
    )

Calculations for B001:

Convexity: 62.0361 years^2

Calculations for B002:

Convexity: 23.8615 years^2

Calculations for B003:

Convexity: 90.0989 years^2

